# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
accident_data_df = pd.read_csv("./data/AviationData.csv", encoding="utf-8")

print(accident_data_df.isna().sum())
print(accident_data_df.info())
accident_data_df.head(10)

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

/tmp/ipykernel_112736/1039024208.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_data_df = pd.read_csv("./data/AviationData.csv", encoding="utf-8")


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
5,20170710X52551,Accident,NYC79AA106,1979-09-17,"BOSTON, MA",United States,42.445277,-70.758333,NaN,NaN,...,NaN,Air Canada,NaN,NaN,1.0,44.0,VMC,Climb,Probable Cause,19-09-2017
6,20001218X45446,Accident,CHI81LA106,1981-08-01,"COTTON, MN",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,IMC,Unknown,Probable Cause,06-11-2001
7,20020909X01562,Accident,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,NaN,NaN,NaN,BLACKBURN AG STRIP,...,Personal,NaN,0.0,0.0,0.0,2.0,VMC,Takeoff,Probable Cause,01-01-1982
8,20020909X01561,Accident,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,NaN,NaN,N58,HANOVER,...,Business,NaN,0.0,0.0,0.0,2.0,IMC,Landing,Probable Cause,01-01-1982
9,20020909X01560,Accident,MIA82DA029,1982-01-01,"JACKSONVILLE, FL",United States,NaN,NaN,JAX,JACKSONVILLE INTL,...,Personal,NaN,0.0,0.0,3.0,0.0,IMC,Cruise,Probable Cause,01-01-1982


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [3]:
accident_data_df['Event.Date'] = pd.to_datetime(accident_data_df['Event.Date'], errors='coerce')

accident_data_df['Event.Date'].head(10)

0   1948-10-24
1   1962-07-19
2   1974-08-30
3   1977-06-19
4   1979-08-02
5   1979-09-17
6   1981-08-01
7   1982-01-01
8   1982-01-01
9   1982-01-01
Name: Event.Date, dtype: datetime64[ns]

In [4]:
accident_data_df['Amateur.Built'].value_counts()

Amateur.Built
No     80312
Yes     8475
Name: count, dtype: int64

In [5]:
accident_data_df['Aircraft.Category'].isna().sum()

np.int64(56602)

In [6]:
accident_data_df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [7]:
accident_data_df['Engine.Type'].value_counts()

Engine.Type
Reciprocating      69530
Turbo Shaft         3609
Turbo Prop          3391
Turbo Fan           2481
Unknown             2051
Turbo Jet            703
Geared Turbofan       12
Electric              10
NONE                   2
LR                     2
Hybrid Rocket          1
UNK                    1
Name: count, dtype: int64

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [8]:
injury_cols = [
    'Total.Fatal.Injuries', 
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured'
]

for col in injury_cols:
    if col in accident_data_df.columns:
        accident_data_df[col] = pd.to_numeric(accident_data_df[col], errors='coerce')

# Creating column for total known injuries (to estimate total people on board)
accident_data_df['Total.Known'] = accident_data_df[injury_cols].sum(axis=1, min_count=1)

# Constructing fatal/serious fraction metric
accident_data_df['Fatal-Serious-Injuries'] = (
    (accident_data_df['Total.Fatal.Injuries'].fillna(0) + accident_data_df['Total.Serious.Injuries'].fillna(0))
    / accident_data_df['Total.Known']
)

accident_data_df['Fatal-Serious-Injuries'] = accident_data_df['Fatal-Serious-Injuries'].replace([float('inf'), -float('inf')], pd.NA)

# Quick check
accident_data_df[['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Known','Fatal-Serious-Injuries']].head(10)

,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Known,Fatal-Serious-Injuries
0,2.0,0.0,2.0,1.0
1,4.0,0.0,4.0,1.0
2,3.0,NaN,3.0,1.0
3,2.0,0.0,2.0,1.0
4,1.0,2.0,3.0,1.0
5,NaN,NaN,45.0,0.0
6,4.0,0.0,4.0,1.0
7,0.0,0.0,2.0,0.0
8,0.0,0.0,2.0,0.0
9,0.0,0.0,3.0,0.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [9]:
accident_data_df['Aircraft.damage'].unique()

array(['Destroyed', 'Substantial', 'Minor', nan, 'Unknown'], dtype=object)

In [10]:
accident_data_df['Aircraft.Destroyed'] = accident_data_df['Aircraft.damage'].eq('Destroyed')

accident_data_df[['Aircraft.Destroyed', 'Aircraft.damage']]

,Aircraft.Destroyed,Aircraft.damage
0,True,Destroyed
1,True,Destroyed
2,True,Destroyed
3,True,Destroyed
4,True,Destroyed
...,...,...
88884,False,NaN
88885,False,NaN
88886,False,Substantial
88887,False,NaN


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [11]:
accident_data_df['Make'].unique().size

8238

In [12]:
accident_data_df['Make'].value_counts()

Make
Cessna               22227
Piper                12029
CESSNA                4922
Beech                 4330
PIPER                 2841
                     ...  
SCOTT TERRY G            1
JAMES R DERNOVSEK        1
ORLICAN S R O            1
ROYSE RALPH L            1
RHINEHART                1
Name: count, Length: 8237, dtype: int64

In [13]:
accident_data_df['Make'] = accident_data_df['Make'].str.upper()

accident_data_df['Make'].value_counts()

Make
CESSNA                   27149
PIPER                    14870
BEECH                     5372
BOEING                    2745
BELL                      2722
                         ...  
MULHOLLAND ROBERT A          1
PIPISTREL D O O              1
ECLIPSE AEROSPACE INC        1
SCHMIDT RANDALL M            1
STEPHEN J HOFFMAN            1
Name: count, Length: 7587, dtype: int64

In [14]:
make_counts = accident_data_df['Make'].value_counts()

common_makes = make_counts[make_counts >= 50].index.to_list()

accident_data_df = accident_data_df[accident_data_df['Make'].isin(common_makes)]

accident_data_df['Make'].value_counts()

Make
CESSNA                         27149
PIPER                          14870
BEECH                           5372
BOEING                          2745
BELL                            2722
                               ...  
LANCAIR                           52
SMITH, TED AEROSTAR               51
BOEING STEARMAN                   51
GRUMMAN AMERICAN AVN. CORP.       50
FLIGHT DESIGN GMBH                50
Name: count, Length: 98, dtype: int64

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [15]:
accident_data_df = accident_data_df.dropna(subset=['Model'])

In [16]:
model_make_duos = accident_data_df.groupby('Model')['Make']
nunique_model_makes = model_make_duos.nunique()
nunique_model_makes = nunique_model_makes[nunique_model_makes > 1]

nunique_model_makes

Model
100                  4
100-180              2
105                  2
109                  2
109A                 2
                    ..
TB20                 2
TB21                 2
THRUSH S2R           3
TIGER MOTH DH 82A    2
UNKNOWN              2
Name: Make, Length: 498, dtype: int64

In [17]:
accident_data_df['plane_type'] = accident_data_df['Make'] + '-' + accident_data_df['Model']

accident_data_df['plane_type']

0                           STINSON-108-3
1                          PIPER-PA24-180
2                             CESSNA-172M
3                            ROCKWELL-112
4                              CESSNA-501
                       ...               
88884                     PIPER-PA-28-151
88885                       BELLANCA-7ECA
88886    AMERICAN CHAMPION AIRCRAFT-8GCBC
88887                         CESSNA-210N
88888                     PIPER-PA-24-260
Name: plane_type, Length: 74627, dtype: object

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [18]:
accident_data_df['Engine.Type'].value_counts()

Engine.Type
Reciprocating      58626
Turbo Shaft         3116
Turbo Prop          2886
Turbo Fan           2220
Unknown             1611
Turbo Jet            565
Geared Turbofan       12
LR                     1
UNK                    1
NONE                   1
Name: count, dtype: int64

In [19]:
accident_data_df['Engine.Type'] = accident_data_df['Engine.Type'].str.replace('UNK', 'Unknown')

accident_data_df['Engine.Type'].value_counts()

Engine.Type
Reciprocating      58626
Turbo Shaft         3116
Turbo Prop          2886
Turbo Fan           2220
Unknown             1612
Turbo Jet            565
Geared Turbofan       12
LR                     1
NONE                   1
Name: count, dtype: int64

In [20]:
accident_data_df['Weather.Condition'].value_counts()

Weather.Condition
VMC    64185
IMC     5601
UNK      802
Unk      231
Name: count, dtype: int64

In [21]:
accident_data_df['Weather.Condition'] = accident_data_df['Weather.Condition'].str.upper()

accident_data_df['Weather.Condition'].value_counts()

Weather.Condition
VMC    64185
IMC     5601
UNK     1033
Name: count, dtype: int64

In [22]:
accident_data_df['Number.of.Engines'].value_counts()

Number.of.Engines
1.0    57842
2.0    10121
0.0      795
3.0      450
4.0      411
8.0        1
Name: count, dtype: int64

In [23]:
accident_data_df['Purpose.of.flight'].value_counts()

Purpose.of.flight
Personal                     39217
Instructional                 9845
Unknown                       6129
Aerial Application            4293
Business                      3753
Positioning                   1451
Other Work Use                1069
Aerial Observation             727
Ferry                          708
Public Aircraft                666
Executive/corporate            478
Skydiving                      177
Flight Test                    165
Banner Tow                      97
External Load                   85
Public Aircraft - Federal       76
Public Aircraft - State         58
Public Aircraft - Local         52
Glider Tow                      41
Firefighting                    33
Air Race show                   30
Air Race/show                   18
Air Drop                        10
PUBS                             3
ASHO                             3
PUBL                             1
Name: count, dtype: int64

In [24]:
accident_data_df['Broad.phase.of.flight'].value_counts()

Broad.phase.of.flight
Landing        13979
Takeoff        10550
Cruise          9044
Maneuvering     6713
Approach        5602
Taxi            1809
Climb           1752
Descent         1666
Go-around       1251
Standing         865
Unknown          454
Other             91
Name: count, dtype: int64

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [25]:
nan_percentages = accident_data_df.isna().sum() / len(accident_data_df) * 100

nan_percentages

Event.Id                   0.000000
Investigation.Type         0.000000
Accident.Number            0.000000
Event.Date                 0.000000
Location                   0.065660
Country                    0.274700
Latitude                  64.700444
Longitude                 64.707144
Airport.Code              43.982741
Airport.Name              40.879306
Injury.Severity            1.184558
Aircraft.damage            3.695713
Aircraft.Category         66.329881
Registration.Number        1.595937
Make                       0.000000
Model                      0.000000
Amateur.Built              0.117920
Number.of.Engines          6.709368
Engine.Type                7.487907
FAR.Description           66.605920
Schedule                  84.670428
Purpose.of.flight          7.292267
Air.carrier               81.814893
Total.Fatal.Injuries      12.684417
Total.Serious.Injuries    13.879695
Total.Minor.Injuries      13.176196
Total.Uninjured            6.064829
Weather.Condition          5

In [26]:
threshold_NaN = 50

cols_to_drop = nan_percentages[nan_percentages >= threshold_NaN].index.to_list()

final_data = accident_data_df.drop(columns=cols_to_drop)

final_data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Known,Fatal-Serious-Injuries,Aircraft.Destroyed,plane_type
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,Fatal(2),Destroyed,...,0.0,0.0,UNK,Cruise,Probable Cause,NaN,2.0,1.0,True,STINSON-108-3
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,Fatal(4),Destroyed,...,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,4.0,1.0,True,PIPER-PA24-180
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,NaN,NaN,Fatal(3),Destroyed,...,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007,3.0,1.0,True,CESSNA-172M
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,Fatal(2),Destroyed,...,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,2.0,1.0,True,ROCKWELL-112
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,Fatal(1),Destroyed,...,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980,3.0,1.0,True,CESSNA-501


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [27]:
final_data.to_csv('data/CleanedAviationData.csv')